In [1]:
import os
import sys
import time
import numpy as np

# Download and install the Python COCO tools from https://github.com/waleedka/coco
# That's a fork from the original https://github.com/pdollar/coco with a bug
# fix for Python 3.
# I submitted a pull request https://github.com/cocodataset/cocoapi/pull/50
# If the PR is merged then use the original repo.

# Note: Edit PythonAPI/Makefile and replace "python" with "python3".
#  
# A quick one liner to install the library 
# !pip install git+https://github.com/waleedka/coco.git#subdirectory=PythonAPI
# import keras.backend as K
# K.clear_session()
# from pycocotools.fakecocoformydata import COCO
import urllib.request
import shutil
from mrcnn.dataset import TreeCountingDataset

2022-05-19 10:12:42.319601: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config

In [3]:
import imgaug as ia
from imgaug import augmenters as iaa
import argparse

In [4]:
ia.seed(1)

# Example batch of images.
# The array has shape (32, 64, 64, 3) and dtype uint8.
seq = iaa.Sometimes(0.5,iaa.Sequential([
    iaa.Fliplr(0.5),
    #Flip/mirror input images horizontally# horizontal flips
    iaa.Flipud(0.5),
    #Flip/mirror input images vertically.
    iaa.Multiply((0.8, 1.2), per_channel=0.5),
    # Multiply all pixels in an image with a specific value, thereby making the image darker or brighter.
    # Multiply 50% of all images with a random value between 0.5 and 1.5 
    # and multiply the remaining 50% channel-wise, i.e. sample one multiplier independently per channel
    iaa.Affine(
       scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        #Scale images to a value of 80 to 120% 
        # of their original size, but do this independently per axis (i.e. sample two values per image)
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        # Translate images by -10 to +10% on x- and y-axis independently
            rotate=(-90, 90),
        # Rotate images by -90 to 90 degrees
#             shear=(-15, 15),
#             cval=(0, 255),
#             mode=ia.ALL
    )
]))


In [5]:
model_size=256
stage_1=180
stage_2=250
stage_3=600
max_gt_instances=20
roi_positive_ratio=0.33
from mrcnn import model as modellib, utils

In [6]:
model_name = "master_model"
train_step = 500
val_step = 100
roi_positive_ratio=0.66
rpn_nms_th=0.7
max_gt_instances = 30

In [7]:
class MappingChallengeConfig(Config):
    """Configuration for training on data in MS COCO format.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = model_name

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Uncomment to train on 8 GPUs (default is 1)
    GPU_COUNT = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # 1 Backgroun + 1 Building
    IMAGE_CHANNEL_COUNT = 3
    STEPS_PER_EPOCH = train_step
    VALIDATION_STEPS = val_step
    # RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    MASK_SHAPE = [28, 28]
    BACKBONE = "resnet101"
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)
    IMAGE_RESIZE_MODE = "square"
    DETECTION_MAX_INSTANCES = 100
    MAX_GT_INSTANCES = max_gt_instances
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    ROI_POSITIVE_RATIO = roi_positive_ratio
    RPN_NMS_THRESHOLD = rpn_nms_th
    # RPN_NMS_THRESHOLD = 0.9
    # ROI_POSITIVE_RATIO = 0.5
    IMAGE_MAX_DIM=model_size+64
    IMAGE_MIN_DIM=model_size+64
    # MEAN_PIXEL = np.array([127.5])
    MEAN_PIXEL = np.array([123.7, 116.8, 103.9])
    IMAGE_CHANNEL_COUNT = 3

In [8]:
config = MappingChallengeConfig()
config.display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  320
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  320
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [320 320   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

In [9]:
data_dir = r"/media/skymap/Data/Building_master_data/data_result"
log_dir = r"/media/skymap/Data/Building_master_data/data_result/logs2_new"

In [10]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=log_dir)
# model_path=model.get_imagenet_weights()
model_path="/media/skymap/Data/Building_master_data/data_result/logs2/master_model20220518T2015/mask_rcnn_master_model_0154.h5"

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

2022-05-19 10:13:07.379373: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-19 10:13:07.380873: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-19 10:13:07.526055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 10:13:07.528914: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6575GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-05-19 10:13:07.528966: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-05-19 10:13:07.601731: I tensorflow/stream_executor/platfo

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [11]:
model.load_weights(model_path, by_name=True)#, exclude='conv1'

2022-05-19 10:13:13.241686: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-19 10:13:13.243961: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 10:13:13.244904: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6575GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-05-19 10:13:13.244953: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.s

Re-starting from epoch 154


In [12]:
dataset_train = TreeCountingDataset()
dataset_train.load_dataset(dataset_dir=os.path.join(data_dir, "train"), load_small=False)
dataset_train.prepare()

loading annotations into memory...
Done (t=3.80s)
creating index...
index created!


In [13]:
dataset_val = TreeCountingDataset()
val_coco = dataset_val.load_dataset(dataset_dir=os.path.join(data_dir, "val"), load_small=False, return_coco=True)
dataset_val.prepare()

loading annotations into memory...
Done (t=1.20s)
creating index...
index created!


In [14]:
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=stage_1,
            layers='heads',
            augment = True ,
            augmentation=None)

Training network heads

Starting at epoch 154. LR=0.001

Checkpoint Path: /media/skymap/Data/Building_master_data/data_result/logs2_new/master_model20220518T2015/mask_rcnn_master_model_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)
Epoch 155/180


2022-05-19 10:14:16.550439: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-05-19 10:14:17.152118: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


  1/500 [..............................] - ETA: 2:24:23 - batch: 0.0000e+00 - size: 2.0000 - loss: 0.7155 - rpn_class_loss: 0.0201 - rpn_bbox_loss: 0.2229 - mrcnn_class_loss: 0.1435 - mrcnn_bbox_loss: 0.3291

2022-05-19 10:14:21.163822: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-05-19 10:14:21.163872: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-05-19 10:14:21.163905: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2022-05-19 10:14:21.185155: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcupti.so.10.1
2022-05-19 10:14:21.287338: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES


  3/500 [..............................] - ETA: 2:22 - batch: 1.0000 - size: 2.0000 - loss: 0.7023 - rpn_class_loss: 0.0269 - rpn_bbox_loss: 0.1962 - mrcnn_class_loss: 0.1872 - mrcnn_bbox_loss: 0.2919     

2022-05-19 10:14:21.405736: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-05-19 10:14:21.439781: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-05-19 10:14:21.448996: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-05-19 10:14:21.482691: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /media/skymap/Data/Building_master_data/data_result/logs2_new/master_model20220518T2015/plugins/profile/2022_05_19_10_14_21
2022-05-19 10:14:21.484610: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /media/skymap/Data/Building_master_data/data_result/logs2_new/master_model20220518T2015/plugins/profile/2022_05_19_10_14_21/skymap-HP-Z800-Workstation.trace.json.gz
2022-05-19 10:14:21.555522: I tensorflow/core/profiler/rpc/client/save_profile.

500/500 [==============================] - 121s 207ms/step - batch: 249.5000 - size: 2.0000 - loss: 0.6139 - rpn_class_loss: 0.0276 - rpn_bbox_loss: 0.2407 - mrcnn_class_loss: 0.1067 - mrcnn_bbox_loss: 0.2388 - val_loss: 0.7904 - val_rpn_class_loss: 0.0293 - val_rpn_bbox_loss: 0.3668 - val_mrcnn_class_loss: 0.1141 - val_mrcnn_bbox_loss: 0.2801
Epoch 156/180
500/500 [==============================] - 94s 187ms/step - batch: 249.5000 - size: 2.0000 - loss: 0.6418 - rpn_class_loss: 0.0295 - rpn_bbox_loss: 0.2529 - mrcnn_class_loss: 0.1151 - mrcnn_bbox_loss: 0.2442 - val_loss: 0.6682 - val_rpn_class_loss: 0.0294 - val_rpn_bbox_loss: 0.2731 - val_mrcnn_class_loss: 0.1173 - val_mrcnn_bbox_loss: 0.2485
Epoch 157/180
500/500 [==============================] - 81s 163ms/step - batch: 249.5000 - size: 2.0000 - loss: 0.6389 - rpn_class_loss: 0.0270 - rpn_bbox_loss: 0.2543 - mrcnn_class_loss: 0.1100 - mrcnn_bbox_loss: 0.2476 - val_loss: 0.8235 - val_rpn_class_loss: 0.0376 - val_rpn_bbox_loss: 0.37

Epoch 178/180
500/500 [==============================] - 84s 169ms/step - batch: 249.5000 - size: 2.0000 - loss: 0.6151 - rpn_class_loss: 0.0270 - rpn_bbox_loss: 0.2419 - mrcnn_class_loss: 0.1120 - mrcnn_bbox_loss: 0.2342 - val_loss: 0.6561 - val_rpn_class_loss: 0.0277 - val_rpn_bbox_loss: 0.2517 - val_mrcnn_class_loss: 0.1210 - val_mrcnn_bbox_loss: 0.2556
Epoch 179/180
500/500 [==============================] - 84s 168ms/step - batch: 249.5000 - size: 2.0000 - loss: 0.6024 - rpn_class_loss: 0.0248 - rpn_bbox_loss: 0.2371 - mrcnn_class_loss: 0.1045 - mrcnn_bbox_loss: 0.2359 - val_loss: 0.8274 - val_rpn_class_loss: 0.0353 - val_rpn_bbox_loss: 0.3892 - val_mrcnn_class_loss: 0.1301 - val_mrcnn_bbox_loss: 0.2728
Epoch 180/180
500/500 [==============================] - 81s 162ms/step - batch: 249.5000 - size: 2.0000 - loss: 0.6241 - rpn_class_loss: 0.0283 - rpn_bbox_loss: 0.2501 - mrcnn_class_loss: 0.1097 - mrcnn_bbox_loss: 0.2361 - val_loss: 0.7162 - val_rpn_class_loss: 0.0300 - val_rpn_bb

In [ ]:
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=stage_2,
            layers='4+',
            augment = True,
            augmentation=None)

Fine tune Resnet stage 4 and up

Starting at epoch 180. LR=0.001

Checkpoint Path: /media/skymap/Data/Building_master_data/data_result/logs2_new/master_model20220518T2015/mask_rcnn_master_model_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          (BatchNorm)
res4c_branch2c         (Conv2D)
bn4c_branch2c          (BatchNorm)
res4d_branch2a         (Conv2D)
bn4d_branch2a          (BatchNorm)
res4d_branch2b         (Conv

2022-05-19 10:55:31.975397: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-05-19 10:55:31.975446: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-05-19 10:55:31.979481: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES


  3/500 [..............................] - ETA: 2:53 - batch: 1.0000 - size: 2.0000 - loss: 1.2754 - rpn_class_loss: 0.0297 - rpn_bbox_loss: 0.2578 - mrcnn_class_loss: 0.1364 - mrcnn_bbox_loss: 0.2138     

2022-05-19 10:55:32.188547: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-05-19 10:55:32.272863: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-05-19 10:55:32.302135: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-05-19 10:55:32.321872: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /media/skymap/Data/Building_master_data/data_result/logs2_new/master_model20220518T2015/plugins/profile/2022_05_19_10_55_32
2022-05-19 10:55:32.323391: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /media/skymap/Data/Building_master_data/data_result/logs2_new/master_model20220518T2015/plugins/profile/2022_05_19_10_55_32/skymap-HP-Z800-Workstation.trace.json.gz
2022-05-19 10:55:32.421668: I tensorflow/core/profiler/rpc/client/save_profile.

500/500 [==============================] - 139s 221ms/step - batch: 249.5000 - size: 2.0000 - loss: 1.5310 - rpn_class_loss: 0.0396 - rpn_bbox_loss: 0.3249 - mrcnn_class_loss: 0.1323 - mrcnn_bbox_loss: 0.2688 - val_loss: 1.4261 - val_rpn_class_loss: 0.0299 - val_rpn_bbox_loss: 0.3132 - val_mrcnn_class_loss: 0.1095 - val_mrcnn_bbox_loss: 0.2605
Epoch 182/250
500/500 [==============================] - 112s 225ms/step - batch: 249.5000 - size: 2.0000 - loss: 1.4413 - rpn_class_loss: 0.0335 - rpn_bbox_loss: 0.2889 - mrcnn_class_loss: 0.1344 - mrcnn_bbox_loss: 0.2639 - val_loss: 1.4740 - val_rpn_class_loss: 0.0289 - val_rpn_bbox_loss: 0.3041 - val_mrcnn_class_loss: 0.1427 - val_mrcnn_bbox_loss: 0.2612
Epoch 183/250
500/500 [==============================] - 107s 214ms/step - batch: 249.5000 - size: 2.0000 - loss: 1.3964 - rpn_class_loss: 0.0328 - rpn_bbox_loss: 0.2864 - mrcnn_class_loss: 0.1228 - mrcnn_bbox_loss: 0.2561 - val_loss: 1.3113 - val_rpn_class_loss: 0.0295 - val_rpn_bbox_loss: 0.

In [ ]:
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE/10,
            epochs=stage_3,
            layers='all',
            augment = True,
            augmentation=None)